# A Demo script showing how to plot FID length for trolley and fixed probes

# The normal setup

In [1]:
import os
import sys
import numpy as np
import pandas as pd
print(np.version.version)
#Set the directory that contains the ROOT data
os.environ['ARTTFSDIR'] = "/Users/bono/Desktop/tempdatadir/run1"
print("data dir = ",os.environ['ARTTFSDIR'])


#add the gm2 pyroot tool to the path (within the shell)
sys.path.append('/Users/bono/Desktop/PythonTool/gm2/lib')
#Bypass using the thisgm2.sh file

#Set BASEDIR and GM2
base_dir = "/Users/bono/Desktop/PythonTool/gm2"

os.environ['BASEDIR'] = base_dir
os.environ['GM2'] = base_dir
print("GM2 = ",os.environ['GM2'])

#append PYTHONPATH
python_path = os.environ['PYTHONPATH'] + ":" + base_dir + "/lib"
os.environ['PYTHONPATH'] = python_path
print("PYTHONPATH = ",os.environ['PYTHONPATH'])

#load the library
import gm2
import matplotlib.pyplot as plt


#Import my functions
import sys
sys.path.append('/Users/bono/Desktop/gm2FieldAnalysis/MyMath')
from avg_rules import trap_avg
sys.path.append('/Users/bono/Desktop/gm2FieldAnalysis/DataSelection')
from treat_data import direct_removal, repeat_averager, spike_removal, phi_selection, degree_of_outlier
from treat_data import spike_removalB
from my_spike import my_spike_removal, get_derivative
sys.path.append('/Users/bono/Desktop/gm2FieldAnalysis/Fitters')
from multipoles import joe_multipoles, my_multipoles

#specify the data dir
data_dir = '/Users/bono/Desktop/gm2FieldAnalysis/RootToText/IO/'

1.16.2
data dir =  /Users/bono/Desktop/tempdatadir/run1
GM2 =  /Users/bono/Desktop/PythonTool/gm2
PYTHONPATH =  /usr/local/Cellar/root/6.16.00/lib/root:/Users/bono/Desktop/PythonTool/gm2/lib
Welcome to JupyROOT 6.16/00


# First the trolley
## The naive way does not, at first, work for variables in branches that differ from the basic one

In [20]:
run = 3956
tr = gm2.Trolley([run])
tr_time, tr_pos, tr_freq = tr.getBasics()
print(tr.getMultipole())
print(tr.getFidLength())
print(tr.getFid()) 

Reading event 8780/8781 99.99%                                                                                                       
loop done: 100%                                    
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 1.09591096e+226 -9.95608566e+051 -1.76228595e-008 -2.89169532e+172
 -4.91148513e+145 -3.27469580e+215  3.79150641e+286 -5.18210863e-133
  8.66736273e-203  2.93305539e+122 -4.50847813e+288 -1.76398155e-059
 -6.62167821e+093 -9.79734482e+221 -1.49722746e-129 -1.09487587e-271
  9.21556173e-235]
[[ 3.25209972e+009  6.16365793e-241 -1.28927412e-071 ... -1.09864804e+278
  -1.03518737e-296 -8.45644541e-254]
 [ 3.79690681e+264  3.46053426e-211 -6.15477333e-053 ...  2.45466975e-161
  -2.26527521e+150 -4.95922593e+225]
 [ 3.15201432e+060 -1.45695258e+004 -3.40228365e-034 ...  4.68618862e+079
   1.50310207e-016  7.34828531e+300]
 ...
 [-2.24632805e-135  4.87859430e-228 -2.11918978e+274 ...  3.41501384e+157
  -8.34351041e+146 -4.88928769e+296]
 [-6.19181449e-282 -3.88658096e+174 -3.33

### You just have to use tr.load() TWICE to get data from the other branches

In [21]:
ev = 10
tr.load(ev)
print( (tr.getFrequency()).shape)
print(tr.getFidLength())
print("~~~~")
print(tr.getMultipole())
print("~~~~")
print(tr.getFid()) 

(17, 5)
[ 1.09591096e+226 -9.95608566e+051 -1.76228595e-008 -2.89169532e+172
 -4.91148513e+145 -3.27469580e+215  3.79150641e+286 -5.18210863e-133
  8.66736273e-203  2.93305539e+122 -4.50847813e+288 -1.76398155e-059
 -6.62167821e+093 -9.79734482e+221 -1.49722746e-129 -1.09487587e-271
  9.21556173e-235]
~~~~
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
~~~~
[[ 3.25209972e+009  6.16365793e-241 -1.28927412e-071 ... -1.09864804e+278
  -1.03518737e-296 -8.45644541e-254]
 [ 3.79690681e+264  3.46053426e-211 -6.15477333e-053 ...  2.45466975e-161
  -2.26527521e+150 -4.95922593e+225]
 [ 3.15201432e+060 -1.45695258e+004 -3.40228365e-034 ...  4.68618862e+079
   1.50310207e-016  7.34828531e+300]
 ...
 [-2.24632805e-135  4.87859430e-228 -2.11918978e+274 ...  3.41501384e+157
  -8.34351041e+146 -4.88928769e+296]
 [-6.19181449e-282 -3.88658096e+174 -3.33963188e-025 ...  9.68217671e-044
   1.08404709e+144  4.13830750e-143]
 [-4.87115473e-193 -7.77525336e-241 -2.26723741e-093 ...  1.61986532e+166
   1.31151870e+122  5.30

## Alternatively, you can use tr.loop(callback) to customize output
## This is the most stable

In [22]:
tr = gm2.Trolley([run])
def callback():
   return [tr.getFrequency(), tr.getFidLength(), tr.getMultipole()]
freq, length, mp = tr.loop(callback)


Reading event 8780/8781 99.99%            
loop done: 100%                                    


### Note that freq now has a shape of (8781, 17, 5). The 5 is for the 5 extraction methods. be careful!

In [24]:
print(freq.shape)
print(length.shape)
print(mp.shape)
print(tr_freq.shape)
print(length)


(8781, 17, 5)
(8781, 17)
(8781, 9)
(8781, 17)
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.00842242e-05 5.02105604e-06 5.02105604e-06 ... 5.02105604e-06
  5.02105604e-06 5.02105604e-06]
 [4.21768707e-05 5.02105604e-06 3.16326531e-04 ... 5.02105604e-06
  5.02105604e-06 5.02105604e-06]
 ...
 [8.57495951e-03 6.74829932e-03 8.17126660e-03 ... 7.14295432e-03
  6.71716877e-03 7.71435050e-03]
 [8.63119534e-03 6.76737933e-03 8.22750243e-03 ... 7.20019436e-03
  6.66093294e-03 7.71435050e-03]
 [8.59403952e-03 6.71013929e-03 8.20842242e-03 ... 7.16203434e-03
  6.71716877e-03 7.71535471e-03]]


# Next, the fixed probe
## It's best to use the loop for the FP


In [28]:
run = 3956
fp = gm2.FixedProbe([run])
def callback():
   return [fp.getFidLength(), 0]
length,dummy = fp.loop(callback) #need a dummy because callback needs to return a list
print(length.shape)
print(length)

Reading event 2630/2634 99.85%   
loop done: 100%                                    
(2634, 378)
[[0.       0.       0.       ... 0.       0.       0.      ]
 [0.00021  0.001187 0.001085 ... 0.001091 0.000299 0.000299]
 [0.000213 0.001196 0.001094 ... 0.001094 0.000297 0.000301]
 ...
 [0.000211 0.001184 0.00108  ... 0.001079 0.000297 0.000303]
 [0.000212 0.001202 0.001089 ... 0.001091 0.000298 0.000299]
 [0.000209 0.001198 0.00108  ... 0.001084 0.000298 0.000301]]


## The non-loop for the fixed probe works just as poorly as it does for the trolley

In [29]:
run = 3956
fp = gm2.FixedProbe([run])
print(fp.getFidLength())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

## This extra step is needed, but it often return nonsense 

In [30]:
ev = 100
fp.load(ev)
print((fp.getFidLength()).shape)
print(fp.getFidLength())

(378,)
[2.140e-04 1.202e-03 1.084e-03 1.070e-04 6.900e-05 5.800e-05 1.200e-04
 3.690e-04 1.253e-03 6.800e-05 1.060e-04 1.510e-04 2.320e-04 6.950e-04
 1.045e-03 8.900e-05 3.700e-05 3.000e-05 4.230e-04 4.630e-04 3.200e-04
 9.900e-05 7.000e-05 8.800e-05 1.070e-04 4.990e-04 3.610e-04 6.800e-05
 5.400e-05 9.400e-05 2.460e-04 8.900e-04 8.430e-04 3.800e-05 6.300e-05
 5.600e-05 5.650e-04 4.650e-04 1.095e-03 1.391e-03 1.122e-03 7.740e-04
 3.450e-04 1.660e-04 4.660e-04 3.360e-04 1.626e-03 5.460e-04 7.450e-04
 5.120e-04 3.330e-04 4.130e-04 5.930e-04 6.910e-04 1.053e-03 3.040e-04
 6.680e-04 8.450e-04 1.719e-03 2.200e-04 2.000e-04 5.260e-04 1.325e-03
 7.220e-04 5.680e-04 7.850e-04 1.549e-03 6.680e-04 4.200e-04 4.760e-04
 3.380e-04 1.500e-04 3.250e-04 2.890e-04 1.570e-04 4.100e-05 8.490e-04
 8.190e-04 2.230e-04 7.590e-04 5.680e-04 2.640e-04 1.812e-03 4.000e-04
 6.440e-04 9.880e-04 6.550e-04 1.129e-03 4.730e-04 5.930e-04 4.570e-04
 2.420e-04 9.940e-04 7.480e-04 6.250e-04 8.400e-04 5.220e-04 8.710e-04